In [1]:
#IMPLEMENTING BINARY DECISION TREES

In [3]:
import graphlab

In [4]:
loans = graphlab.SFrame('lending-club-data.gl/')

[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: /tmp/graphlab_server_1481657606.log


This non-commercial license of GraphLab Create for academic use is assigned to hashokkumar92@gmail.com and will expire on November 22, 2017.


In [5]:
#Formatting the target variable to include safe and risky loans
loans['safe_loans'] = loans['bad_loans'].apply(lambda x : +1 if x==0 else -1)
loans = loans.remove_column('bad_loans')

In [6]:
#Extracting few features to use in our prediction
features = ['grade',              # grade of the loan
            'term',               # the term of the loan
            'home_ownership',     # home_ownership status: own, mortgage or rent
            'emp_length',         # number of years of employment
           ]
target = 'safe_loans'
loans = loans[features + [target]]

In [11]:
loans

grade,term,home_ownership,emp_length,safe_loans
B,36 months,RENT,10+ years,1
C,60 months,RENT,< 1 year,-1
C,36 months,RENT,10+ years,1
C,36 months,RENT,10+ years,1
A,36 months,RENT,3 years,1
E,36 months,RENT,9 years,1
F,60 months,OWN,4 years,-1
B,60 months,RENT,< 1 year,-1
C,60 months,OWN,5 years,1
B,36 months,OWN,10+ years,1


In [12]:
#SubSampling to make sure we have balanced classes
safe_loans_raw = loans[loans[target] == 1]
risky_loans_raw = loans[loans[target] == -1]

# Since there are less risky loans than safe loans, find the ratio of the sizes
# and use that percentage to undersample the safe loans.
percentage = len(risky_loans_raw)/float(len(safe_loans_raw))
safe_loans = safe_loans_raw.sample(percentage, seed = 1)
risky_loans = risky_loans_raw
loans_data = risky_loans.append(safe_loans)

print "Percentage of safe loans                 :", len(safe_loans) / float(len(loans_data))
print "Percentage of risky loans                :", len(risky_loans) / float(len(loans_data))
print "Total number of loans in our new dataset :", len(loans_data)

Percentage of safe loans                 : 0.502236174422
Percentage of risky loans                : 0.497763825578
Total number of loans in our new dataset : 46508


In [14]:
#CATEGORICAL VARIABLE - BINARY TRANSFORMATION
for feature in features:
    loans_data_one_hot_encoded = loans_data[feature].apply(lambda x: {x: 1})    
    loans_data_unpacked = loans_data_one_hot_encoded.unpack(column_name_prefix=feature)
    
    # Change None's to 0's
    for column in loans_data_unpacked.column_names():
        loans_data_unpacked[column] = loans_data_unpacked[column].fillna(0)

    loans_data.remove_column(feature)
    loans_data.add_columns(loans_data_unpacked)

In [15]:
loans_data

safe_loans,grade.A,grade.B,grade.C,grade.D,grade.E,grade.F,grade.G,term. 36 months,term. 60 months
-1,0,0,1,0,0,0,0,0,1
-1,0,0,0,0,0,1,0,0,1
-1,0,1,0,0,0,0,0,0,1
-1,0,0,1,0,0,0,0,1,0
-1,0,1,0,0,0,0,0,1,0
-1,0,1,0,0,0,0,0,1,0
-1,0,1,0,0,0,0,0,1,0
-1,0,0,1,0,0,0,0,1,0
-1,0,0,0,1,0,0,0,0,1
-1,1,0,0,0,0,0,0,1,0


In [17]:
features = loans_data.column_names()
features.remove('safe_loans')  # Remove the response variable
print features

print "Number of features (after binarizing categorical variables) = %s" % len(features)


['grade.A', 'grade.B', 'grade.C', 'grade.D', 'grade.E', 'grade.F', 'grade.G', 'term. 36 months', 'term. 60 months', 'home_ownership.MORTGAGE', 'home_ownership.OTHER', 'home_ownership.OWN', 'home_ownership.RENT', 'emp_length.1 year', 'emp_length.10+ years', 'emp_length.2 years', 'emp_length.3 years', 'emp_length.4 years', 'emp_length.5 years', 'emp_length.6 years', 'emp_length.7 years', 'emp_length.8 years', 'emp_length.9 years', 'emp_length.< 1 year', 'emp_length.n/a']
Number of features (after binarizing categorical variables) = 25


In [18]:
loans_data.head(n=1)

Columns:
	safe_loans	int
	grade.A	int
	grade.B	int
	grade.C	int
	grade.D	int
	grade.E	int
	grade.F	int
	grade.G	int
	term. 36 months	int
	term. 60 months	int
	home_ownership.MORTGAGE	int
	home_ownership.OTHER	int
	home_ownership.OWN	int
	home_ownership.RENT	int
	emp_length.1 year	int
	emp_length.10+ years	int
	emp_length.2 years	int
	emp_length.3 years	int
	emp_length.4 years	int
	emp_length.5 years	int
	emp_length.6 years	int
	emp_length.7 years	int
	emp_length.8 years	int
	emp_length.9 years	int
	emp_length.< 1 year	int
	emp_length.n/a	int

Rows: 1

Data:
+------------+---------+---------+---------+---------+---------+---------+---------+
| safe_loans | grade.A | grade.B | grade.C | grade.D | grade.E | grade.F | grade.G |
+------------+---------+---------+---------+---------+---------+---------+---------+
|     -1     |    0    |    0    |    1    |    0    |    0    |    0    |    0    |
+------------+---------+---------+---------+---------+---------+---------+---------+
+-----------------+-----------------+-------------------------+----------------------+
| term. 36 months | term. 60 months | home_ownership.MORTGAGE | home_ownership.OTHER |
+-----------------+-----------------+-------------------------+----------------------+
|        0        |        1        |            0            |          0           |
+-----------------+-----------------+-------------------------+----------------------+
+--------------------+---------------------+-------------------+----------------------+
| home_ownership.OWN | home_ownership.RENT | emp_length.1 year | emp_length.10+ years |
+--------------------+---------------------+-------------------+----------------------+
|         0          |          1          |         0         |          0           |
+--------------------+---------------------+-------------------+----------------------+
+--------------------+--------------------+--------------------+--------------------+-----+
| emp_length.2 years | emp_length.3 years | emp_length.4 years | emp_length.5 years | ... |
+--------------------+--------------------+--------------------+--------------------+-----+
|         0          |         0          |         0          |         0          | ... |
+--------------------+--------------------+--------------------+--------------------+-----+
[1 rows x 26 columns]

In [19]:
print "Total number of grade.A loans : %s" % loans_data['grade.A'].sum()
print "Expexted answer               : 6422"

Total number of grade.A loans : 6422
Expexted answer               : 6422


In [20]:
#TRAINING/TESTING DATA SPLIT
train_data, test_data = loans_data.random_split(.8, seed=1)


In [23]:
#DECISION TREE IMPLEMENTATION


#Compute the number of misclassified examples at an intermediate node

def intermediate_node_mistakes(labels_node):
    if len(labels_node) == 0:
        return 0
    
    num_positive = (labels_node == +1).sum()
    num_negative = (labels_node == -1).sum()
    
    return num_negative if num_positive > num_negative else num_positive

In [30]:
def best_split(data, features, target):
    best_feature = None
    best_error = 10
    
    num_datapoints = float(len(data))
    
    for feature in features:
        left_split = data[data[feature] == 0]
        right_split = data[data[feature] == 1]
        
        #calculating the misclassified examples
        left_mistakes = intermediate_node_mistakes(left_split[target])
        right_mistakes = intermediate_node_mistakes(right_split[target])
        
        error = (left_mistakes + right_mistakes)/num_datapoints
        
        if error < best_error:
            best_feature = feature
            best_error = error
        
    return best_feature
    

In [31]:
best_split(train_data,features, 'safe_loans')

'term. 36 months'

In [33]:
#Building the Tree

#Creating the leaf
def create_leaf(target_values):
    #leaf node
    leaf = {'splitting_feature': None,
           'left': None,
           'right':None,
           'is_leaf':True}
    
    num_plus_ones = len(target_values[target_values]== +1)
    num_minus_ones = len(target_values[target_values]== -1)
    
    if num_plus_ones > num_minus_ones:
        leaf['prediction'] = +1
    else:
        leaf['prediction'] = -1
    
    return leaf

In [40]:
#Creating decision Tree
def decision_tree_create(data, features, target, current_depth = 0, max_depth= 10):
    remaining_features = features[:]
    target_values = data[target]
    
    #1. Chck if there are mistakes in current node
    if intermediate_node_mistakes(target_values)== 0:
        print "Stopping condition 1 reached."
        return create_leaf(target_values)
    
    #2. Check if there are remaining features to split on.
    if remaining_features == []:
        print "Stopping condition 2 reached."
        return create_leaf(target_values)
    
    #3.Additional stopping condition for max depth.
    if current_depth>max_depth:
        print "Reached maximum depth.Stooping for now"
        return create_leaf(target_values)
    
    #Finding the best splitting feature
    splitting_feature = best_split(data, features, target)

    left_split = data[data[splitting_feature] == 0]
    right_split = data[data[splitting_feature] == 1]
    remaining_features.remove(splitting_feature)

    print "Split on feature %s. (%s, %s)" % (\
                      splitting_feature, len(left_split), len(right_split))

    # Create a leaf node if the split is "perfect"
    if len(left_split) == len(data):
        print "Creating leaf node."
        return create_leaf(left_split[target])
    if len(right_split) == len(data):
        print "Creating leaf node."
        return create_leaf(right_split[target])
    
    #Repeat on right and left subtrees
    left_tree = decision_tree_create(left_split, remaining_features, target, current_depth + 1, max_depth)        
    
    right_tree = decision_tree_create(right_split, remaining_features, target, current_depth + 1, max_depth) 

    return {'is_leaf'          : False, 
            'prediction'       : None,
            'splitting_feature': splitting_feature,
            'left'             : left_tree, 
            'right'            : right_tree}
        

In [41]:
#Recursive function to count the nodes
def count_nodes(tree):
    if tree['is_leaf']:
        return 1
    return 1 + count_nodes(tree['left']) + count_nodes(tree['right'])

In [42]:
# Make sure to cap the depth at 6 by using max_depth = 6
my_decision_tree = decision_tree_create(train_data, features, 'safe_loans', max_depth = 6)

Split on feature term. 36 months. (9223, 28001)
Split on feature grade.A. (9122, 101)
Split on feature grade.B. (8074, 1048)
Split on feature grade.C. (5884, 2190)
Split on feature grade.D. (3826, 2058)
Split on feature grade.E. (1693, 2133)
Split on feature home_ownership.OTHER. (1692, 1)
Reached maximum depth.Stooping for now
Stopping condition 1 reached.
Split on feature grade.F. (2133, 0)
Creating leaf node.
Split on feature grade.E. (2058, 0)
Creating leaf node.
Split on feature grade.D. (2190, 0)
Creating leaf node.
Split on feature emp_length.5 years. (969, 79)
Split on feature grade.C. (969, 0)
Creating leaf node.
Split on feature home_ownership.MORTGAGE. (34, 45)
Split on feature grade.C. (34, 0)
Creating leaf node.
Split on feature grade.C. (45, 0)
Creating leaf node.
Split on feature emp_length.n/a. (96, 5)
Split on feature emp_length.< 1 year. (85, 11)
Split on feature grade.B. (85, 0)
Creating leaf node.
Split on feature grade.B. (11, 0)
Creating leaf node.
Split on featur

In [43]:
my_decision_tree

{'is_leaf': False,
 'left': {'is_leaf': False,
  'left': {'is_leaf': False,
   'left': {'is_leaf': False,
    'left': {'is_leaf': False,
     'left': {'is_leaf': False,
      'left': {'is_leaf': False,
       'left': {'is_leaf': True,
        'left': None,
        'prediction': -1,
        'right': None,
        'splitting_feature': None},
       'prediction': None,
       'right': {'is_leaf': True,
        'left': None,
        'prediction': -1,
        'right': None,
        'splitting_feature': None},
       'splitting_feature': 'home_ownership.OTHER'},
      'prediction': None,
      'right': {'is_leaf': True,
       'left': None,
       'prediction': -1,
       'right': None,
       'splitting_feature': None},
      'splitting_feature': 'grade.E'},
     'prediction': None,
     'right': {'is_leaf': True,
      'left': None,
      'prediction': -1,
      'right': None,
      'splitting_feature': None},
     'splitting_feature': 'grade.D'},
    'prediction': None,
    'right': {'is_

In [44]:
#MAKING PREDICTIONS WITH DECISION TREE
def classify(tree, x, annotate = False):   
    # if the node is a leaf node.
    if tree['is_leaf']:
        if annotate: 
            print "At leaf, predicting %s" % tree['prediction']
        return tree['prediction'] 
    else:
        # split on feature.
        split_feature_value = x[tree['splitting_feature']]
        if annotate: 
            print "Split on %s = %s" % (tree['splitting_feature'], split_feature_value)
        if split_feature_value == 0:
            return classify(tree['left'], x, annotate)
        else:
            return classify(tree['right'], x, annotate)

In [45]:
print 'Predicted class: %s ' % classify(my_decision_tree, test_data[0])


Predicted class: -1 


In [46]:
classify(my_decision_tree, test_data[0], annotate=True)


Split on term. 36 months = 0
Split on grade.A = 0
Split on grade.B = 0
Split on grade.C = 0
Split on grade.D = 1
At leaf, predicting -1


-1

In [49]:

def evaluate_classification_error(tree, data):
    # Apply the classify(tree, x) to each row
    prediction = data.apply(lambda x: classify(tree, x))    
    #Calculate the classification error and return it
    num_of_mistakes = (prediction != data[target]).sum()/float(len(data))
    return num_of_mistakes

In [50]:
evaluate_classification_error(my_decision_tree, test_data)


0.496553209823352

In [51]:
def print_stump(tree, name = 'root'):
    split_name = tree['splitting_feature'] # split_name is something like 'term. 36 months'
    if split_name is None:
        print "(leaf, label: %s)" % tree['prediction']
        return None
    split_feature, split_value = split_name.split('.')
    print '                       %s' % name
    print '         |---------------|----------------|'
    print '         |                                |'
    print '         |                                |'
    print '         |                                |'
    print '  [{0} == 0]               [{0} == 1]    '.format(split_name)
    print '         |                                |'
    print '         |                                |'
    print '         |                                |'
    print '    (%s)                         (%s)' \
        % (('leaf, label: ' + str(tree['left']['prediction']) if tree['left']['is_leaf'] else 'subtree'),
           ('leaf, label: ' + str(tree['right']['prediction']) if tree['right']['is_leaf'] else 'subtree'))

In [53]:
print print_stump(my_decision_tree)
print print_stump(my_decision_tree['left'], my_decision_tree['splitting_feature'])
print print_stump(my_decision_tree['left']['left'], my_decision_tree['left']['splitting_feature'])
print print_stump(my_decision_tree['right'], my_decision_tree['splitting_feature'])



                       root
         |---------------|----------------|
         |                                |
         |                                |
         |                                |
  [term. 36 months == 0]               [term. 36 months == 1]    
         |                                |
         |                                |
         |                                |
    (subtree)                         (subtree)
None
                       term. 36 months
         |---------------|----------------|
         |                                |
         |                                |
         |                                |
  [grade.A == 0]               [grade.A == 1]    
         |                                |
         |                                |
         |                                |
    (subtree)                         (subtree)
None
                       grade.A
         |---------------|----------------|
         |          